# Jonathan Halverson
# Saturday, March 4, 2017
# Write out the ufc.com data to a latex table

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/ufc_dot_com_fighter_data_CLEAN_28Feb2017.csv', header=0)
df.head(3)

,Name,Nickname,Active,Record,Age,Height,Weight,Reach,LegReach,OutOf,From,College,Degree,Summary
0,Danny Abbadi,The Assassin,0,NaN,33.0,71.0,185.0,NaN,NaN,NaN,Amman Jordan,NaN,NaN,Striking and kicks
1,Tank Abbott,Tank,0,NaN,51.0,72.0,250.0,NaN,NaN,NaN,"Orange, California USA",NaN,NaN,"Punching power, strength"
2,Shamil Abdurakhimov,Abrek,1,17-4-0,35.0,75.0,235.0,76.0,41.0,Dagestan Russia,Dagestan Russia,NaN,NaN,Wrestling and boxing


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1897 entries, 0 to 1896
Data columns (total 14 columns):
Name        1897 non-null object
Nickname    1048 non-null object
Active      1897 non-null int64
Record      533 non-null object
Age         1509 non-null float64
Height      1555 non-null float64
Weight      1557 non-null float64
Reach       870 non-null float64
LegReach    565 non-null float64
OutOf       1112 non-null object
From        1709 non-null object
College     173 non-null object
Degree      157 non-null object
Summary     1483 non-null object
dtypes: float64(5), int64(1), object(8)
memory usage: 207.6+ KB


In [4]:
df = df[df.Active == 1][['Name', 'Record', 'Age', 'Weight', 'Height', 'Reach', 'LegReach']].reset_index(drop=True)
df.head(3)

,Name,Record,Age,Weight,Height,Reach,LegReach
0,Shamil Abdurakhimov,17-4-0,35.0,235.0,75.0,76.0,41.0
1,Ricardo Abreu,5-1-0,32.0,185.0,71.0,73.0,NaN
2,Jessica Aguilar,19-5-0,NaN,115.0,64.0,NaN,NaN


In [5]:
rev = df.sort_index(ascending=False).reset_index(drop=True)
rev.head(3)

,Name,Record,Age,Weight,Height,Reach,LegReach
0,Cat Zingano,9-2-0,34.0,135.0,66.0,68.0,38.0
1,Tiequan Zhang,18-4-0,38.0,155.0,68.0,NaN,NaN
2,Elizeu Zaleski dos Santos,16-5-0,30.0,170.0,71.0,73.0,41.0


### Note that Series.replace considers the entire content of the cell in determing if there is a match. While Series.str.replace take a regex expression.

In [6]:
cmb = pd.merge(df, rev, how='inner', left_index=True, right_index=True)
cmb.Record_x = cmb.Record_x.str.replace('-', '--')
cmb.Record_y = cmb.Record_y.str.replace('-', '--')
cmb = cmb.fillna(0)
cmb = cmb.astype({'Age_x':int, 'Weight_x':int, 'Height_x':int, 'Reach_x':int, 'LegReach_x':int})
cmb = cmb.astype({'Age_y':int, 'Weight_y':int, 'Height_y':int, 'Reach_y':int, 'LegReach_y':int})
cmb.columns = ['Name', 'Record', 'Age', 'Wt.', 'Ht.', 'Rh.', 'Lg.', 'Name', 'Record', 'Age', 'Wt.','Ht.', 'Rh.', 'Lg.']
cmb.head(3)

,Name,Record,Age,Wt.,Ht.,Rh.,Lg.,Name,Record,Age,Wt.,Ht.,Rh.,Lg.
0,Shamil Abdurakhimov,17--4--0,35,235,75,76,41,Cat Zingano,9--2--0,34,135,66,68,38
1,Ricardo Abreu,5--1--0,32,185,71,73,0,Tiequan Zhang,18--4--0,38,155,68,0,0
2,Jessica Aguilar,19--5--0,0,115,64,0,0,Elizeu Zaleski dos Santos,16--5--0,30,170,71,73,41


In [7]:
import math
thres = math.ceil(df.shape[0] / 2.0)
iofile = 'ufc_table.tex'
cmb[:int(thres)].to_latex(iofile, index=False, na_rep='', longtable=True)

In [8]:
with open(iofile) as f:
     lines = f.readlines()
with open(iofile, 'w') as f:
     for line in lines:
          line = line.replace('llrrrrrllrrrrr','llrrrrr||llrrrrr')
          line = line.replace('\multicolumn{3}{r}','\multicolumn{14}{c}')
          line = line.replace('Continued on next page', 'Active UFC Fighters --- J. D. Halverson --- 2--28--2017')
          f.write(line)